In [73]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-01-06 19:08:19--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275513 (269K) [application/zip]
Saving to: ‘WiC_dataset.zip.1’

WiC_dataset.zip.1   100%[===================>] 269.06K  --.-KB/s    in 0.03s   

2023-01-06 19:08:19 (8.20 MB/s) - ‘WiC_dataset.zip.1’ saved [275513/275513]



In [74]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall()

In [75]:
import pandas as pd
import io
import spacy
nlp = spacy.load("en_core_web_sm")

In [76]:
df = pd.read_csv("/content/dev/dev.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"])

In [77]:
df.head()

,lemma,pos,index,sent1,sent2
0,board,N,2-2,Room and board .,He nailed boards across the windows .
1,circulate,V,0-4,Circulate a rumor .,This letter is being circulated among the facu...
2,hook,V,0-1,Hook a fish .,"He hooked a snake accidentally , and was so sc..."
3,recreation,N,1-9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...
4,domesticity,N,4-6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...


In [78]:
x = df['index'].tolist()

In [79]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

In [80]:
df.loc[:,'ind1'] = ind1
df.loc[:,'ind2'] = ind2

In [81]:
df

,lemma,pos,index,sent1,sent2,ind1,ind2
0,board,N,2-2,Room and board .,He nailed boards across the windows .,2,2
1,circulate,V,0-4,Circulate a rumor .,This letter is being circulated among the facu...,0,4
2,hook,V,0-1,Hook a fish .,"He hooked a snake accidentally , and was so sc...",0,1
3,recreation,N,1-9,For recreation he wrote poetry and solved cros...,Drug abuse is often regarded as a form of recr...,1,9
4,domesticity,N,4-6,Making a hobby of domesticity .,A royal family living in unpretentious domesti...,4,6
...,...,...,...,...,...,...,...
633,base,N,9-0,He told the painter that he wanted a yellow ba...,Bases include oxides and hydroxides of metals ...,9,0
634,power,N,5-6,He needed a lot of power to hit the ball out o...,The mysterious presence of an evil power .,5,6
635,portmanteau,N,8-5,` brunch ' is a well - known portmanteau .,` motel ' is a portmanteau word made by combin...,8,5
636,promise,V,1-1,I promise you my best effort .,I promised somebody my time .,1,1


In [82]:
df1 = df[['lemma', 'pos', 'sent1', 'ind1']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind']

In [83]:
df2 = df[['lemma', 'pos', 'sent2', 'ind2']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind']

In [84]:
df_final = pd.concat([df1, df2])

In [85]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [86]:
df_final["indexes_target_token"] = df_final.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)

In [87]:
df_final

,lemma,pos,context_tokenized,ind,indexes_target_token
0,board,N,Room and board .,2,9:14
1,circulate,V,Circulate a rumor .,0,0:9
2,hook,V,Hook a fish .,0,0:4
3,recreation,N,For recreation he wrote poetry and solved cros...,1,4:14
4,domesticity,N,Making a hobby of domesticity .,4,18:29
...,...,...,...,...,...
633,base,N,Bases include oxides and hydroxides of metals ...,0,0:5
634,power,N,The mysterious presence of an evil power .,6,35:40
635,portmanteau,N,` motel ' is a portmanteau word made by combin...,5,15:26
636,promise,V,I promised somebody my time .,1,2:10


In [88]:
m = df1.context_tokenized[2]

In [89]:
m[0:4]

'Hook'

In [90]:
df_final = df_final.reset_index(drop=True)

In [91]:
df_final

,lemma,pos,context_tokenized,ind,indexes_target_token
0,board,N,Room and board .,2,9:14
1,circulate,V,Circulate a rumor .,0,0:9
2,hook,V,Hook a fish .,0,0:4
3,recreation,N,For recreation he wrote poetry and solved cros...,1,4:14
4,domesticity,N,Making a hobby of domesticity .,4,18:29
...,...,...,...,...,...
1271,base,N,Bases include oxides and hydroxides of metals ...,0,0:5
1272,power,N,The mysterious presence of an evil power .,6,35:40
1273,portmanteau,N,` motel ' is a portmanteau word made by combin...,5,15:26
1274,promise,V,I promised somebody my time .,1,2:10


In [92]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [93]:
df_final["indexes_target_sentence"] = df_final["context_tokenized"].apply(get_indices_of_sent)

In [94]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [95]:
df_final['indexes_target_sentence_tokenized'] = df_final["context_tokenized"].apply(get_len_tok)

In [96]:
df_final.head()

,lemma,pos,context_tokenized,ind,indexes_target_token,indexes_target_sentence,indexes_target_sentence_tokenized
0,board,N,Room and board .,2,9:14,0:16,0:4
1,circulate,V,Circulate a rumor .,0,0:9,0:19,0:4
2,hook,V,Hook a fish .,0,0:4,0:13,0:4
3,recreation,N,For recreation he wrote poetry and solved cros...,1,4:14,0:61,0:10
4,domesticity,N,Making a hobby of domesticity .,4,18:29,0:31,0:6


In [97]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [98]:
df_final['context_lemmatized'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [99]:

df_final['context_pos'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [100]:
df_final.head()

,lemma,pos,context_tokenized,ind,indexes_target_token,indexes_target_sentence,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,board,N,Room and board .,2,9:14,0:16,0:4,room and board .,NOUN CCONJ NOUN PUNCT
1,circulate,V,Circulate a rumor .,0,0:9,0:19,0:4,circulate a rumor .,VERB DET NOUN PUNCT
2,hook,V,Hook a fish .,0,0:4,0:13,0:4,hook a fish .,VERB DET NOUN PUNCT
3,recreation,N,For recreation he wrote poetry and solved cros...,1,4:14,0:61,0:10,for recreation he write poetry and solve cross...,ADP NOUN PRON VERB NOUN CCONJ VERB ADJ NOUN PUNCT
4,domesticity,N,Making a hobby of domesticity .,4,18:29,0:31,0:6,make a hobby of domesticity .,VERB DET NOUN ADP NOUN PUNCT


In [101]:
df_final['date'] = " "
df_final['grouping'] = 1
df_final['description'] = " "
df_final['context'] = df_final['context_tokenized']
df_final['identifier'] = " "
df_final['indexes_target_token_tokenized'] = df_final['ind']

In [102]:
final_df = df_final[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]

In [103]:
final_df

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,context_tokenized,indexes_target_token_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,board,N,,1,,,Room and board .,9:14,0:16,Room and board .,2,0:4,room and board .,NOUN CCONJ NOUN PUNCT
1,circulate,V,,1,,,Circulate a rumor .,0:9,0:19,Circulate a rumor .,0,0:4,circulate a rumor .,VERB DET NOUN PUNCT
2,hook,V,,1,,,Hook a fish .,0:4,0:13,Hook a fish .,0,0:4,hook a fish .,VERB DET NOUN PUNCT
3,recreation,N,,1,,,For recreation he wrote poetry and solved cros...,4:14,0:61,For recreation he wrote poetry and solved cros...,1,0:10,for recreation he write poetry and solve cross...,ADP NOUN PRON VERB NOUN CCONJ VERB ADJ NOUN PUNCT
4,domesticity,N,,1,,,Making a hobby of domesticity .,18:29,0:31,Making a hobby of domesticity .,4,0:6,make a hobby of domesticity .,VERB DET NOUN ADP NOUN PUNCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,base,N,,1,,,Bases include oxides and hydroxides of metals ...,0:5,0:59,Bases include oxides and hydroxides of metals ...,0,0:10,basis include oxide and hydroxide of metal and...,NOUN VERB NOUN CCONJ NOUN ADP NOUN CCONJ NOUN ...
1272,power,N,,1,,,The mysterious presence of an evil power .,35:40,0:42,The mysterious presence of an evil power .,6,0:8,the mysterious presence of an evil power .,DET ADJ NOUN ADP DET ADJ NOUN PUNCT
1273,portmanteau,N,,1,,,` motel ' is a portmanteau word made by combin...,15:26,0:75,` motel ' is a portmanteau word made by combin...,5,0:18,` motel ' be a portmanteau word make by combin...,PUNCT NOUN PUNCT AUX DET NOUN NOUN VERB ADP VE...
1274,promise,V,,1,,,I promised somebody my time .,2:10,0:29,I promised somebody my time .,1,0:6,I promise somebody my time .,PRON VERB PRON PRON NOUN PUNCT


In [104]:
#c = final_df.context.tolist()

In [105]:
#y = []
#for m in c:
 # y.append(m.split(" "))

In [106]:
#lengthh = []
#for i in y:
 #   lengthh.append(len(i))

In [107]:
#final_df.loc[:,'lengthh'] = lengthh

In [108]:
#d = final_df.context_pos.tolist()

In [109]:
#z = []
#for m in d:
 # z.append(m.split(" "))

In [110]:
#lengthh2 = []
#for i in z:
 #   lengthh2.append(len(i))

In [111]:
#final_df.loc[:,'lengthh2'] = lengthh2

In [112]:
final_df

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,context_tokenized,indexes_target_token_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,board,N,,1,,,Room and board .,9:14,0:16,Room and board .,2,0:4,room and board .,NOUN CCONJ NOUN PUNCT
1,circulate,V,,1,,,Circulate a rumor .,0:9,0:19,Circulate a rumor .,0,0:4,circulate a rumor .,VERB DET NOUN PUNCT
2,hook,V,,1,,,Hook a fish .,0:4,0:13,Hook a fish .,0,0:4,hook a fish .,VERB DET NOUN PUNCT
3,recreation,N,,1,,,For recreation he wrote poetry and solved cros...,4:14,0:61,For recreation he wrote poetry and solved cros...,1,0:10,for recreation he write poetry and solve cross...,ADP NOUN PRON VERB NOUN CCONJ VERB ADJ NOUN PUNCT
4,domesticity,N,,1,,,Making a hobby of domesticity .,18:29,0:31,Making a hobby of domesticity .,4,0:6,make a hobby of domesticity .,VERB DET NOUN ADP NOUN PUNCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,base,N,,1,,,Bases include oxides and hydroxides of metals ...,0:5,0:59,Bases include oxides and hydroxides of metals ...,0,0:10,basis include oxide and hydroxide of metal and...,NOUN VERB NOUN CCONJ NOUN ADP NOUN CCONJ NOUN ...
1272,power,N,,1,,,The mysterious presence of an evil power .,35:40,0:42,The mysterious presence of an evil power .,6,0:8,the mysterious presence of an evil power .,DET ADJ NOUN ADP DET ADJ NOUN PUNCT
1273,portmanteau,N,,1,,,` motel ' is a portmanteau word made by combin...,15:26,0:75,` motel ' is a portmanteau word made by combin...,5,0:18,` motel ' be a portmanteau word make by combin...,PUNCT NOUN PUNCT AUX DET NOUN NOUN VERB ADP VE...
1274,promise,V,,1,,,I promised somebody my time .,2:10,0:29,I promised somebody my time .,1,0:6,I promise somebody my time .,PRON VERB PRON PRON NOUN PUNCT


In [113]:
#final_df['lengthh2'].equals(final_df['lengthh'])

In [114]:
m = final_df.context[1274]

In [115]:
y = m.split(" ")